# Class Activation Map

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing import image
# import Keras's functional api
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions

In [2]:
model = tf.keras.models.load_model('models/plant_village_VGG19_trained_01.h5')

In [3]:
img = image.load_img('dataset/PlantVillage/PlantVillage-Dataset/raw/color/Corn_(maize)___Common_rust_/RS_Rust 1609.JPG', 
                    target_size= (224,224)
                    )

In [46]:
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, 0)
x = tf.convert_to_tensor(x)
x.shape , x.dtype

(TensorShape([1, 224, 224, 3]), tf.float32)

In [52]:
model.layers[0].summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
class_weights = model.layers[0].layers[-2].get_weights()[0]

In [41]:
# init_layer = model.layers[0].get_layer('input_1')
init_layer = model.layers[0]
init_layer.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [33]:
x.shape
#init_layer.input.shape

(1, 224, 224, 3)

In [14]:
final_conv_layer = model.layers[0].get_layer('block5_conv4')
final_conv_layer.output

<tf.Tensor 'block5_conv4/Identity:0' shape=(None, 14, 14, 512) dtype=float32>

In [42]:
get_output = tf.keras.backend.function([init_layer.input], [final_conv_layer.output, model.layers[-1].output])

In [50]:
[conv_outputs, predictions] = get_output.x

AttributeError: 'EagerExecutionFunction' object has no attribute 'x'

In [ ]:
def visualize_class_activation_map(model_path, img_path, output_path):
        model = load_model(model_path)
        original_img = cv2.imread(img_path, 1)
        width, height, _ = original_img.shape

        #Reshape to the network input shape (3, w, h).
        img = np.array([np.transpose(np.float32(original_img), (2, 0, 1))])
        
        #Get the 512 input weights to the softmax.
        class_weights = model.layers[-1].get_weights()[0]
        final_conv_layer = get_output_layer(model, "conv5_3")
        get_output = K.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
        [conv_outputs, predictions] = get_output([img])
        conv_outputs = conv_outputs[0, :, :, :]

        #Create the class activation map.
        cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[1:3])
        for i, w in enumerate(class_weights[:, 1]):
                cam += w * conv_outputs[i, :, :]
        print "predictions", predictions
        cam /= np.max(cam)
        cam = cv2.resize(cam, (height, width))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam < 0.2)] = 0
        img = heatmap*0.5 + original_img
        cv2.imwrite(output_path, img)